# MAP PREPARATION FOR SPATIALLY-EXPLICIT NEUTRAL MODELS 

This file requires
- scotese shape files for continental boundaries at different times (280, 320 and 340 mya)
- info_per_pcoords.csv for location data for each fragment (output of R script)

# This file outputs
- Raster files for each interval for continental extents
- Raster files for sampling regimes across the world
- Shapefiles for all sites at each interval

In [1]:
import math
import numpy as np
import os
import pandas as pd
import shutil
import sys
sys.path.append("../")
from data_preparation import rasterize, extract_from_shapefile, \
	create_shapefile_from_points, find_distance_lat_long, randomly_clear_landscape, add_masks
from pycoalescence import Map

In [2]:
# Data set up
# set this to the path to the data directory. Everything else should work from that
# data_directory = "/Volumes/Seagate 3TB/Paleo/Data/"
data_directory = "/run/media/sam/Media/Paleo/Data/"
occ_sq_output_dir = os.path.join(data_directory, "paleo_maps")
# the list of folders containing the shape files to convert to raster
csv_directory = "../../MainSimulationR/input"

In [3]:
interval_csv = pd.read_csv(os.path.join(csv_directory, "interval_data.csv"))
interval_csv.interval = [x.lower() for x in interval_csv.interval]

In [24]:
# This extracts the continental shapes from the shapefiles into a new file, and then converts it to a
# raster.
main_dir = os.path.join(data_directory, "paleo_maps", "main")
if not os.path.exists(main_dir):
    os.mkdir(main_dir)
for folder in set(interval_csv.map_file):
	file_path = os.path.join(data_directory, "scotese", folder, "wcnt.shp")
	file_path_altered = os.path.join(data_directory, "scotese", folder, "wcnt_altered.shp")
	if not os.path.exists(file_path):
		raise IOError("file {} does not exist".format(file_path))
	# Drop all features from the shapefile which don't match "CM" field
	extract_from_shapefile(file_path, file_path_altered, field="TYPE", field_value="CM")
	# Now convert our altered shapefile to a raster
	output_path = os.path.join(main_dir, "{}.tif".format(folder))
	rasterize(file_path_altered, output_path, pixel_size=0.01)

In [26]:
# Remove the wasteful file
for file in os.listdir(main_dir):
    os.remove(os.path.join(main_dir, file))
os.rmdir(main_dir)

In [4]:
# Import the fragment coordinates from csv.
info_per_pcoord = pd.read_csv(os.path.join(csv_directory, "info_per_pcoord_occ.csv"))
info_per_pcoord['lat'] = info_per_pcoord.coord_lat
info_per_pcoord['long'] = info_per_pcoord.coord_long

In [5]:
# Create the shape file from these points and the spatial sampling masks,
#  which are = 1 at our fossil sites, and 0 everywhere else.
# These sampling masks will be updated later on to contain the sampling proportion at each site,
# not just a binary mask.
for interval in info_per_pcoord['interval'].unique():
	for tet_group in info_per_pcoord['tetrapod_group'].unique():
		tmp_df = info_per_pcoord[((info_per_pcoord.interval == interval) &
								  (info_per_pcoord.tetrapod_group == tet_group))]
		# the mask files for defining our spatial sampling
		mask_shp = os.path.join(occ_sq_output_dir,
								"pointsmask_{}_{}.shp".format(interval.replace(" ", "_").lower(),
															  tet_group))
		mask_raster = os.path.join(occ_sq_output_dir,
								"paleomask_occ_sq_{}_{}.tif".format(interval.replace(" ", "_").lower(),
															 tet_group))
		create_shapefile_from_points(tmp_df, mask_shp)
		# use our mask to create a binary raster map
		rasterize(mask_shp, mask_raster, pixel_size=0.01, field="prop_ind")

In [6]:
interval_list = set([x.replace(" ", "_").lower() for x in info_per_pcoord["interval"]])
tetrapod_g_list = set(info_per_pcoord["tetrapod_group"])

In [7]:
# Calculate the origins for each of the time points
mask_origins = {}
for interval in interval_list:
	for tet_group in tetrapod_g_list:
		mask_raster = os.path.join(occ_sq_output_dir,
								"paleomask_occ_sq_{}_{}.tif".format(interval, tet_group))
		paleo_mask = Map(mask_raster)
		_, _, _,_, x_res, y_res, ulx_mask, uly_mask = paleo_mask.get_dimensions()
		mask_origins[(interval, tet_group)] = [uly_mask, ulx_mask, y_res, x_res]

In [8]:
# Lower case the info_per_pcoord
info_per_pcoord["interval"] = [x.lower() for x in info_per_pcoord["interval"]]
interval_max_ind = info_per_pcoord.groupby(["interval", "tetrapod_group"],
										   squeeze=True)["individuals_total_sq"].max().reset_index()
interval_max_ind.to_csv(os.path.join(csv_directory, "max_individuals_sq.csv"))

In [9]:
max_density_dict = {}
all_intervals = []
all_tetrapod_groups = []
for i, row in interval_max_ind.iterrows():
	all_intervals.append(row.interval)
	all_tetrapod_groups.append(row.tetrapod_group)
	max_density_dict[(row.interval, row.tetrapod_group)] = row.individuals_total_sq
all_intervals = set(all_intervals)
all_tetrapod_groups = set(all_tetrapod_groups)


In [10]:
from collections import OrderedDict
OrderedDict(max_density_dict)

OrderedDict([(('artinskian', 'amniote'), 75),
             (('artinskian', 'amphibian'), 75),
             (('asselian', 'amniote'), 30),
             (('asselian', 'amphibian'), 30),
             (('bashkirian', 'amniote'), 1),
             (('bashkirian', 'amphibian'), 115),
             (('gzhelian', 'amniote'), 35),
             (('gzhelian', 'amphibian'), 35),
             (('kasimovian', 'amniote'), 35),
             (('kasimovian', 'amphibian'), 16),
             (('kungurian', 'amniote'), 75),
             (('kungurian', 'amphibian'), 85),
             (('moscovian', 'amniote'), 16),
             (('moscovian', 'amphibian'), 105),
             (('sakmarian', 'amniote'), 30),
             (('sakmarian', 'amphibian'), 75)])

In [11]:
# Create the fragment csv for defining fragments within the simulation
# This creates one fragment csv for each interval
# We now need to create our masks again, altering the values of the shapefile to match our fragment coordinates
# This seems a bit stupid, but because fragment coordinates can't be calculated until the mask file
# origin is known, this is the best way
if not os.path.exists(os.path.join(data_directory, "configs")):
	os.mkdir(os.path.join(data_directory, "configs"))
for interval in interval_list:
    for tet_group in tetrapod_g_list:
        columns = ['fragment', 'x_min', 'y_min', 'x_max', 'y_max', 'no_individuals']
        rows = [x for x in range(0, info_per_pcoord.shape[0], 1)]
        fragment_locations = pd.DataFrame(columns=columns, index=rows)
        fragment_map = Map(file=os.path.join(occ_sq_output_dir,
                                             "paleomask_occ_sq_{}_{}.tif".format(interval, tet_group)))
        fragment_map.open()
        subset_df = info_per_pcoord[(info_per_pcoord["interval"] == interval) &
                                     (info_per_pcoord["tetrapod_group"]== tet_group)]
        if len(subset_df.index) == 0:
            continue
        for index, row in subset_df.iterrows():
            if row.interval.replace(" ", "_").lower() != interval:
                continue
            modifier = 0
            # # Weirdly one of the fragments is put above instead of below the geo-coord - change it here
            # if row.collection_ref == "174420":
            # 	modifier = -1
            fragment_locations.fragment[index] = row.fragment
            offsets = find_distance_lat_long(mask_origins[(interval, tet_group)][0],
                                             mask_origins[(interval, tet_group)][1], row.lat, row.long, 
                                             res=mask_origins[(interval, tet_group)][3])
            fragment_locations.x_min[index] = int(math.floor(offsets[1])) 
            fragment_locations.x_max[index] = int(math.floor(offsets[1] + 1))
            fragment_locations.y_min[index] = int(math.floor(offsets[0])) 
            fragment_locations.y_max[index] = int(math.floor(offsets[0] + 1)) + modifier
            fragment_locations.no_individuals[index] = row.individuals_total_sq
            this_max_ind = interval_max_ind[(interval_max_ind["interval"] == interval) & 
                                            (interval_max_ind["tetrapod_group"] == tet_group)].individuals_total_sq
            # Alter the fragment mask with updating the sampling values.  ***** add this back in if needed
#             try:
            # Round up the sampling proportion to the nearest whole number (for the minimum density parameter)
            min_density_parameter = 25
            expected_value = math.ceil(min_density_parameter*row.individuals_total_sq/this_max_ind)/min_density_parameter
            fragment_map.data[fragment_locations.y_min[index], fragment_locations.x_min[index]] = expected_value
#             except:
#                 continue
        fragment_locations = fragment_locations[pd.notnull(fragment_locations['x_min'])]
        fragment_locations.to_csv(os.path.join(data_directory, "configs", "fragments_occ_{}_{}.csv".format(interval,
                                                                                                           tet_group)),
                                  index=False, header=False)
        fragment_map.write()

In [28]:
# Fix the fossil sites that are actually in the sea according to the paleomap.
for interval in interval_list:
	interval_map_path = os.path.join(occ_sq_output_dir, "{}.tif".format(interval))
	interval_map = Map(interval_map_path)
	for tet_group in tetrapod_g_list:
		sample_map_path = os.path.join(occ_sq_output_dir,
								  "paleomask_occ_sq_{}_{}.tif".format(interval, tet_group))
		sample_map = Map(sample_map_path)
		sample_map.open()
		subset = np.array(sample_map.data > 0.0).astype(int)
		x, y = sample_map.get_x_y()
		x_off, y_off = sample_map.calculate_offset(interval_map)
		density = np.maximum(interval_map.get_subset(x_off, y_off, x, y, no_data_value=0),
						 subset)
		interval_map.write_subset(density, x_off, y_off)

In [31]:
# Now double check that these all make sense
for interval in interval_list:
    for tet_group in tetrapod_g_list:
        df = pd.read_csv(os.path.join(data_directory, "configs", "fragments_occ_{}_{}.csv".format(interval, tet_group)),
                         header=None)
        df.columns = ['fragment', 'x_min', 'y_min', 'x_max', 'y_max', 'no_individuals']
        mask_raster = Map(os.path.join(occ_sq_output_dir,
                                       "paleomask_occ_sq_{}_{}.tif".format(interval, tet_group)))
        mask_value = mask_raster.get_cached_subset(0, 0, 1, 1)
        this_max_ind = interval_max_ind[(interval_max_ind["interval"] == interval) & 
                                            (interval_max_ind["tetrapod_group"] == tet_group)].individuals_total_sq
        for index, row in df.iterrows():
            # print(row.fragment)
            mask_value = mask_raster.get_cached_subset(row.x_min, row.y_min, 1, 1)
            expected_value = math.ceil(25*row.no_individuals/this_max_ind)/25
            if not math.isclose(mask_value[0][0], expected_value, rel_tol=1e-2):
                print("Mask value not correct: {} != {}"
                      " for location {}, {} - fragment {}".format(mask_value[0][0],
                                                                  row.no_individuals/this_max_ind,
                                                                  row.x_min,
                                                                  row.y_min,
                                                                  row.fragment))
print("All maps verified")

All maps verified


In [21]:
math.ceil(25*1/100)/25

0.04

In [64]:
# Now verify the samplemaps
for interval in interval_list:
	interval_map_path = os.path.join(occ_sq_output_dir, "{}.tif".format(interval))
	interval_map = Map(interval_map_path)
	for tet_group in tetrapod_g_list:
		sample_map_path = os.path.join(occ_sq_output_dir,
                                       "paleomask_occ_sq_{}_{}.tif".format(interval, tet_group))
		sample_map = Map(sample_map_path)
		sample_map.open()
		subset = sample_map.data == 0.0
		x, y = sample_map.get_x_y()
		offset_x, offset_y = sample_map.calculate_offset(interval_map)
		density = np.ma.array(interval_map.get_subset(offset_x, offset_y, x, y, no_data_value=0),
							  mask=subset)
		tot_subset = np.sum(np.invert(subset))
		tot_density = np.ma.sum(density)
		if tot_subset != tot_density:
			print("Total from subset: {}".format(tot_subset))
			print("Total from density: {}".format(tot_density))
			raise ValueError("Zero density in sampled region found in {}, {}.".format(interval, tet_group))
print("All maps verified")

All maps verified


### Generate the fragmented landscapes

Generate 20, 40 and 80% fragmented landscapes for times after 305mya.

In [65]:
if not os.path.exists(os.path.join(data_directory, "paleo_maps", "fragmented")):
	os.mkdir(os.path.join(data_directory, "paleo_maps", "fragmented"))
fragmented_intervals = []
for proportion_cover in [0.2, 0.4, 0.8]:
	for index, row in interval_csv.iterrows():
		if row["midpoint"] < 307:
			interval = row["interval"]
			print("Interval: {}".format(interval))
			fragmented_intervals.append(interval)
			fragmented_map = os.path.join(data_directory, "paleo_maps", "fragmented",
										  "{}_{}_fragmented.tif".format(interval, proportion_cover))
			if not os.path.exists(fragmented_map):
				randomly_clear_landscape(os.path.join(occ_sq_output_dir, "{}.tif".format(interval)),
										 fragmented_map, proportion_cover)
fragmented_intervals = set(fragmented_intervals)

Interval: kasimovian


KeyboardInterrupt: 

In [48]:
# Fix the fossil sites that are actually in the sea according to the paleomap,
# or have been randomly removed
for proportion_cover in [0.2, 0.4, 0.8]:
	print(proportion_cover)
	for interval in interval_list:
		if interval in fragmented_intervals:
			interval_map_path = os.path.join(data_directory, "paleo_maps", "fragmented",
											 "{}_{}_fragmented.tif".format(interval, proportion_cover))
			interval_map = Map(interval_map_path)
			for tet_group in tetrapod_g_list:
				sample_map_path = os.path.join(occ_sq_output_dir,
                                               "paleomask_occ_sq_{}_{}.tif".format(interval, tet_group))
				sample_map = Map(sample_map_path)
				sample_map.open()
				subset = np.array(sample_map.data > 0.0).astype(int)
				x, y = sample_map.get_x_y()
				x_off, y_off = sample_map.calculate_offset(interval_map)
				density = np.maximum(interval_map.get_subset(x_off, y_off, x, y, no_data_value=0),
								 subset)
				interval_map.write_subset(density, x_off, y_off)

0.2
0.4
0.8


In [49]:
# Now verify the samplemaps
for proportion_cover in [0.2, 0.4, 0.8]:
	for interval in interval_list:
		if interval in fragmented_intervals:
			interval_map_path = os.path.join(data_directory, "paleo_maps", "fragmented",
											 "{}_{}_fragmented.tif".format(interval,
																		   proportion_cover))
			interval_map = Map(interval_map_path)
			for tet_group in tetrapod_g_list:
				sample_map_path = os.path.join(occ_sq_output_dir,
                                               "paleomask_occ_sq_{}_{}.tif".format(interval, tet_group))
				sample_map = Map(sample_map_path)
				sample_map.open()
				subset = sample_map.data == 0.0
				x, y = sample_map.get_x_y()
				offset_x, offset_y = sample_map.calculate_offset(interval_map)
				density = np.ma.array(interval_map.get_subset(offset_x, offset_y, x, y, no_data_value=0),
									  mask=subset)
				tot_subset = np.sum(np.invert(subset))
				tot_density = np.ma.sum(density)
				if tot_subset != tot_density:
					print("Total from subset: {}".format(tot_subset))
					print("Total from density: {}".format(tot_density))
					raise ValueError("Zero density in sampled region found in {}, {}"
									 " with {} proportion cover.".format(interval, tet_group, proportion_cover))
print("All maps verified")

All maps verified
